<a href="https://colab.research.google.com/github/ahmedelsayed16112002/Projects/blob/main/Decision_Tree_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df =  pd.read_csv('/content/car_evaluation.csv')

In [3]:
df.head()

,vhigh,vhigh.1,2,2.1,small,low,unacc
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [4]:
df.shape

(1727, 7)

In [5]:
col_name = ['buying','maint','doors','persons','lug_boot','safety','class']
df.columns = col_name
df.columns

Index(['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class'], dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1727 non-null   object
 1   maint     1727 non-null   object
 2   doors     1727 non-null   object
 3   persons   1727 non-null   object
 4   lug_boot  1727 non-null   object
 5   safety    1727 non-null   object
 6   class     1727 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


Summary of variables¶  
There are 7 variables in the dataset. All the variables are of categorical data type.  
These are given by buying, maint, doors, persons, lug_boot, safety and class.
class is the target variable  

In [8]:
df['class'].value_counts()

,count
class,
unacc,1209
acc,384
good,69
vgood,65


# preprocessing

In [11]:
df.isnull().sum()

,0
buying,0
maint,0
doors,0
persons,0
lug_boot,0
safety,0
class,0


In [13]:
df.duplicated().sum()

0

# Split data into separate training and test set ¶


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], test_size=0.33, random_state=0)
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

Shape of X_train:  (1157, 6)
Shape of X_test:  (570, 6)


#  Feature Engineering ¶


In [16]:
X_train.dtypes

,0
buying,object
maint,object
doors,object
persons,object
lug_boot,object
safety,object


In [19]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
X_train = enc.fit_transform(X_train)
X_test = enc.transform(X_test)
X_train = pd.DataFrame(X_train,columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])
X_test = pd.DataFrame(X_test,columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])


In [22]:
X_test.head()

,buying,maint,doors,persons,lug_boot,safety
0,1.0,3.0,0.0,2.0,1.0,0.0
1,3.0,0.0,0.0,1.0,0.0,0.0
2,0.0,2.0,0.0,0.0,2.0,2.0
3,3.0,2.0,1.0,0.0,0.0,2.0
4,1.0,2.0,3.0,0.0,0.0,1.0


#  Decision Tree Classifier with criterion gini index ¶


In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import GridSearchCV
max_depth = [2,3,4,5,6,7,8,9,10]
min_samples_split = [2,3,4,5,6,7,8,9,10]
dt = DecisionTreeClassifier(criterion= 'gini' ,random_state=0)
grid_search = GridSearchCV(estimator=dt,param_grid=dict(max_depth=max_depth,min_samples_split=min_samples_split),cv=10)
grid_search.fit(X_train,y_train)
print("Best parameters: ",grid_search.best_params_)
print("Best score: ",grid_search.best_score_)

Best parameters:  {'max_depth': 10, 'min_samples_split': 2}
Best score:  0.9498575712143928


In [24]:
accuracy_score  = grid_search.score(X_test,y_test)
print(accuracy_score)


0.968421052631579


In [25]:
classification_report  = classification_report(y_test,grid_search.predict(X_test))
print(classification_report)

              precision    recall  f1-score   support

         acc       0.93      0.95      0.94       131
        good       0.88      0.64      0.74        22
       unacc       0.99      0.99      0.99       391
       vgood       0.96      1.00      0.98        26

    accuracy                           0.97       570
   macro avg       0.94      0.89      0.91       570
weighted avg       0.97      0.97      0.97       570



# Visualize decision-trees¶


In [28]:
import graphviz
dot_data = export_graphviz(grid_search.best_estimator_,out_file=None,filled=True,rounded=True)
graph = graphviz.Source(dot_data)
graph.render("car_evaluation")


'car_evaluation.pdf'

# checking for underfitting and overfitting

In [29]:
accuracy_score = grid_search.score(X_train,y_train)
print(accuracy_score)
accuracy_score = grid_search.score(X_test,y_test)
print(accuracy_score)

0.9835782195332757
0.968421052631579


# Decision Tree Classifier with criterion entropy ¶


In [34]:
CLF_DT = DecisionTreeClassifier(criterion='entropy',random_state=0)
Max_depth = [2,3,4,5,6,7,8,9,10]
Min_samples_split = [2,3,4,5,6,7,8,9,10]
grid_search = GridSearchCV(estimator=CLF_DT,param_grid=dict(max_depth=Max_depth,min_samples_split=Min_samples_split),cv=10)
grid_search.fit(X_train,y_train)
print("Best parameters: ",grid_search.best_params_)
print("Best score: ",grid_search.best_score_)


Best parameters:  {'max_depth': 10, 'min_samples_split': 3}
Best score:  0.9654197901049477


In [35]:
accuracy_score = grid_search.score(X_train,y_train)
print(accuracy_score)
accuracy_score = grid_search.score(X_test,y_test)
print(accuracy_score)

0.9878997407087294
0.9614035087719298


In [36]:
dot_data = export_graphviz(grid_search.best_estimator_,out_file=None,filled=True,rounded=True)
graph = graphviz.Source(dot_data)
graph.render("car_evaluation")

'car_evaluation.pdf'